In [5]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [18]:
# jobkorea 검색 결과 크롤링

resp = requests.get("http://www.jobkorea.co.kr/Search/?stext=데이터분석&tabType=recruit&Page_No=1")

In [19]:
print( resp.status_code )
print( resp.text[:200] )

200
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="utf-8" />
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=e


In [20]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [21]:
# 채용정보 목록에서 채용정보 항목 가져오기
lis = soup.select('div.recruit-info li[data-gno]') # 선택자[attr-name] : attr-name 속성이 있는 요소, 선택자
# lis[0]

In [22]:
lis

[]

In [17]:
rows = []
for li in lis:
    cname = li.select_one(".post-list-corp a").get_text().strip()
    title = li.select_one('.post-list-info a').get_text().strip()
    exp = li.select_one('.post-list-info p.option .exp').get_text().strip()
    edu = li.select_one('.post-list-info p.option .edu').get_text().strip()
    loc = li.select_one('.post-list-info p.option .loc.long').get_text().strip() # 두 개의 class 정보는 .class1.class2
    # print(cname, " / ", title, " / ", edu)
    rows.append([cname, title, exp, edu, loc])

# rows

In [19]:
# 페이징이 있는 웹사이트의 여러 페이지에서 데이터 수집

import time

keyword = '하둡'
page_no = 1

rows = []

while True:

    # print("processing page {0}".format(page_no))

    resp = requests.get("http://www.jobkorea.co.kr/Search/?stext={0}&tabType=recruit&Page_No={1}".format(keyword, page_no))

    soup = BeautifulSoup(resp.text, 'html.parser')
    
    lis = soup.select('div.recruit-info li[data-gno]')

    if lis == None or len(lis) == 0: # 마지막페이지 + 1인 조건
        print("end of data")
        break

    for li in lis:
        cname = li.select_one(".post-list-corp a").get_text().strip()
        title = li.select_one('.post-list-info a').get_text().strip()
        exp = li.select_one('.post-list-info p.option .exp').get_text().strip()
        edu = li.select_one('.post-list-info p.option .edu')
        edu = edu if edu == None else edu.get_text().strip()        
        loc = li.select_one('.post-list-info p.option .loc.long').get_text().strip()
        rows.append([cname, title, exp, edu, loc])

    page_no += 1 # page_no = page_no + 1

    time.sleep(1)

end of data


In [20]:
print( len(rows) )
rows[:10]

238


[['카카오페이', '[기술플랫폼실] 빅데이터, 하둡(Hadoop) 플랫폼 운영 엔지니어 영입', '경력', None, '경기 성남시'],
 ['누리꿈소프트㈜',
  '[수도권,정규직] 하둡 플랫폼 구축 및 유지보수 인원 구인합니다.',
  '경력3년↑',
  '학력무관',
  '서울 강남구'],
 ['아이티아이즈',
  'DB설치구성 엔지니어 / 퍼블릭 클라우드 운영자 / 하둡 설치엔지니어 모집',
  '신입·경력2년↑',
  '초대졸↑',
  '서울 영등포구'],
 ['엔에이치엔 주식회사', '[NHN] 2021년 (신입) 전문연구요원 모집', '신입', '석사↑', '경기 성남시'],
 ['㈜유비케어', 'GC녹십자그룹 가족사 상반기 공개채용', '신입·경력', '초대졸↑', '서울 영등포구'],
 ['롯데멤버스㈜',
  '[롯데멤버스] 2021 상반기 경력채용 (제휴컨설팅 / 데이터분석 / 데이터거버넌스 / 디지털마케팅 / 정보보호 / 솔루션사업)',
  '경력3년↑',
  '초대졸↑',
  '서울 중구'],
 ['현대엠엔소프트㈜',
  '신입/경력사원 공개채용(S/W개발,품질,기획/디자인,경영지원 부문)',
  '신입·경력',
  '대졸↑',
  '서울 용산구'],
 ['㈜그리드위즈', '각 부문 신입 및 경력직원 채용', '신입·경력2년↑', '학력무관', '경기 성남시'],
 ['한국직업전문학교',
  '[4차산업 IT과정 - 국비무료] 디지털컨버젼스 응용SW 개발 (JAVA,Python) 모집',
  '신입·경력',
  '학력무관',
  '경기 전지역 외'],
 ['아이비케이투자증권㈜', '2021년 IBK투자증권 경력사원 채용', '경력', '학력무관', '경기 전지역 외']]

In [13]:
pd.DataFrame(data=rows, columns=['cname', 'title', 'exp', 'edu', 'loc'])

cname     0
title     0
exp       0
edu      10
loc       0
dtype: int64

In [2]:
import pymysql # 데이터베이스 연동 모듈 준비

In [3]:
# 1. 연결하기
conn = pymysql.connect(host='localhost',
                       database='pydemodb',
                       user='root',
                       password='Pa$$w0rd',
                       charset='utf8')

# 2. 명령하기
with conn.cursor() as cursor:
    sql = """
        create table tbl_recruit(
            idx int primary key auto_increment,
            cname varchar(100) not null,
            title varchar(1000) not null,
            exp varchar(100) null,
            edu varchar(100) null,
            loc varchar(100) null
        )
    """
    cursor.execute(sql)

conn.close()

In [21]:
# 1. 연결하기
conn = pymysql.connect(host='localhost',
                       database='pydemodb',
                       user='root',
                       password='Pa$$w0rd',
                       charset='utf8')

# 2. 명령하기
with conn.cursor() as cursor:
    sql = """
        insert into tbl_recruit (cname, title, exp, edu, loc) 
        values (%s, %s, %s, %s, %s)
    """
    # for numbers in numbers_list:
    #     cursor.execute(sql, numbers)

    cursor.executemany(sql, rows)

    conn.commit()

conn.close()

['카카오페이', '[기술플랫폼실] 빅데이터, 하둡(Hadoop) 플랫폼 운영 엔지니어 영입', '경력', nan, '경기 성남시']
['누리꿈소프트㈜', '[수도권,정규직] 하둡 플랫폼 구축 및 유지보수 인원 구인합니다.', '경력3년↑', '학력무관', '서울 강남구']
['아이티아이즈', 'DB설치구성 엔지니어 / 퍼블릭 클라우드 운영자 / 하둡 설치엔지니어 모집', '신입·경력2년↑', '초대졸↑', '서울 영등포구']
['엔에이치엔 주식회사', '[NHN] 2021년 (신입) 전문연구요원 모집', '신입', '석사↑', '경기 성남시']
['㈜유비케어', 'GC녹십자그룹 가족사 상반기 공개채용', '신입·경력', '초대졸↑', '서울 영등포구']
['롯데멤버스㈜', '[롯데멤버스] 2021 상반기 경력채용 (제휴컨설팅 / 데이터분석 / 데이터거버넌스 / 디지털마케팅 / 정보보호 / 솔루션사업)', '경력3년↑', '초대졸↑', '서울 중구']
['현대엠엔소프트㈜', '신입/경력사원 공개채용(S/W개발,품질,기획/디자인,경영지원 부문)', '신입·경력', '대졸↑', '서울 용산구']
['㈜그리드위즈', '각 부문 신입 및 경력직원 채용', '신입·경력2년↑', '학력무관', '경기 성남시']
['한국직업전문학교', '[4차산업 IT과정 - 국비무료] 디지털컨버젼스 응용SW 개발 (JAVA,Python) 모집', '신입·경력', '학력무관', '경기 전지역 외']
['아이비케이투자증권㈜', '2021년 IBK투자증권 경력사원 채용', '경력', '학력무관', '경기 전지역 외']
['㈜무하유', '[카피킬러] 2021년 상반기 개발부문 신입/경력사원 채용', '신입·경력', '학력무관', '서울 성동구']
['주식회사 여기어때컴퍼니', '[여기어때] 데이터 엔지니어 (DBA/DW구축) 채용', '경력3년↑', '학력무관', '서울 강남구']
['카카오페이', '[카카오페이] 데이터 엔지니어 영입', '경력3년↑', '학력무관', '경기 성남시']
['㈜